<a href="https://colab.research.google.com/github/OoJackoO/MMAI894_Deep-Learning-Course-Project/blob/dev/Com2_Load_Tomato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 

Data Source: 

https://www.kaggle.com/datasets/cookiefinder/tomato-disease-multiple-sources  

Training set: train folder 

Validation and Test set: valid folder 

Reference: https://cs230.stanford.edu/blog/split/  

Random seed: 24 

In [ ]:
# Import modules
# Add modules as needed
import numpy as np
import os
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

from keras.utils import to_categorical
from keras import layers, regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, BatchNormalization, LayerNormalization
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/MMAI_894/Team_Assignment/'
#!kaggle datasets download -d cookiefinder/tomato-disease-multiple-sources --path "/content/drive/MyDrive/MMAI_894/Team_Assignment/Dataset/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# This function is just to get a list of dirs for the visualization and EDA functions
def define_paths(dir):
    filepaths = []
    labels = []
    #images = []
    folds = os.listdir(dir)
    print("The unique images labels are:", folds)
    for fold in folds:
        foldpath = os.path.join(dir, fold)
        filelist = os.listdir(foldpath)
        print(foldpath)
        print(len(filelist),"files")
        for file in filelist:
            fpath = os.path.join(foldpath, file)
            filepaths.append(fpath)
            labels.append(fold)
    filepaths_array = np.array(filepaths)
    labels_array = np.array(labels)

    return filepaths_array, labels_array

data_dir = ('/content/drive/MyDrive/MMAI_894/Team_Assignment/Dataset/tomato-disease-multiple-sources_Data/train')
filepaths_array, labels_array = define_paths(data_dir)
print(filepaths_array.shape)
print(labels_array.shape)
print(filepaths_array[-1])
print(labels_array[-1])


In [ ]:
def visualize(filepaths_array,labels_array):
  num_images = len(filepaths_array)
  fig, ax = plt.subplots(nrows=1, ncols=num_images, figsize=(20,20))
  print(num_images)
# Display each image on a separate axis
  for i, images_array in enumerate(filepaths_array):
      #print(filepaths_array[i])
      image = mpimg.imread(filepaths_array[i])
      height, width, channels = image.shape
      ax[i].imshow(image)
      print(height, width, channels)
      #ax[i].set_xticks([])
      #ax[i].set_yticks([])
      ax[i].set_title(labels_array[i])

stop = len(filepaths_array)
start = stop-6

visualize(filepaths_array[start:stop], labels_array[start:stop])


NameError: ignored

In [ ]:
def data_distribution(labels_array):
  # Get the count of each label
  unique_labels, counts = np.unique(labels_array, return_counts=True)

# Use a bar chart to show the distribution of labels
  bars = plt.bar(np.arange(len(counts)), counts)
  plt.xticks(np.arange(len(counts)), unique_labels, rotation=90)
  plt.tick_params(labeltop=True, labelbottom=False)
  plt.xlabel('Class')
  plt.ylabel('Count')
  plt.bar_label(bars, labels=counts, label_type='edge', fontsize=10)
  plt.show()

data_distribution(labels_array)


NameError: ignored

In [ ]:
from tqdm import tqdm

# This functions builds a generator -> splits images into batches, normalizes each image, reshapes each image to the specified size, extracts the category classes, one hot encoded the catergories
def loadandsplit_images(dir):

  data_dir = dir
  data_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)
  
  random_seed = 24
  batch_size = 128

  train_generator = data_datagen.flow_from_directory(
      data_dir,
      target_size = (256,256),
      batch_size = batch_size,
      class_mode = 'categorical',
      subset='training',
      seed=random_seed
  )
  
  val_generator = data_datagen.flow_from_directory(
      data_dir,
      target_size = (256,256),
      batch_size = batch_size,
      class_mode = 'categorical',
      subset='validation',
      seed=random_seed
  )

  total_samples = train_generator.samples + val_generator.samples
  print("\nTotal number of samples:", total_samples)
  print("Number of training image samples:", train_generator.samples)
  print("Number of validation image samples:", val_generator.samples)
  print("\nX_train % split:", train_generator.samples / total_samples*100,"%")
  print("X_val % split:", val_generator.samples / total_samples*100,"%")
  print("Batch size:", batch_size)


  return train_generator, val_generator

data_dir = ('/content/drive/MyDrive/MMAI_894/Team_Assignment/Dataset/tomato-disease-multiple-sources_Data/train')
train_gen, val_gen = loadandsplit_images(data_dir)

Found 12714 images belonging to 11 classes.
Found 12709 images belonging to 11 classes.

Total number of samples: 25423
Number of training image samples: 12714
Number of validation image samples: 12709

X_train % split: 50.009833615230306 %
X_val % split: 49.990166384769694 %
Batch size: 128


In [ ]:
def build_model():
    # TODO: build the model, 
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
    

    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size = (4, 4), activation = 'relu', input_shape=(256, 256, 3),strides = (1,1), padding = "same", name="Convolution_Layer1"))
    model.add(layers.MaxPool2D(2,2))
    model.add(Conv2D(filters=64, kernel_size=(4, 4), activation = 'relu', strides=(1,1), padding="same", name="Convolution_Layer2"))
    model.add(layers.MaxPool2D(2,2))
    model.add(Conv2D(64, (4, 4), activation = 'relu', strides=(1,1), padding="same",name="Convolution_Layer3"))
    model.add(layers.Flatten(name="Flatten_layer"))

    
    #Layer 1 with 64 nodes and relu activation. Input will get calcualted by Keras
    # A single Dense layer looked to give suffecient performance as compared to adding two layers i.e. not too much benefit was observed in this case by having two dense layers
    #model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001), name="Layer_1"))
    #Add dropout of 0.30 (this number was arrived upon via hyperparameter tuning)
    model.add(Dropout(0.30))
    #Added an Batch normalization layer to help improve performance - this didn't help
    #model.add(BatchNormalization())
    #Output with softmax activation and 11 out out layers corresponding to the output classes
    model.add(Dense(11, activation='softmax',  name="Output"))
    #Lets see how the model looks
    model.summary()   
    print(model.get_layer("Flatten_layer").output_shape)

    return model

def compile_model(model):
    # TODO: compile the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    #Lets try different optimizers as well as learning rates to see if it helps
    optimizer = Adam()
    #optimizer = Adam(learning_rate=0.001)
    #optimizer = RMSprop()
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = build_model()
model = compile_model(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Convolution_Layer1 (Conv2D)  (None, 256, 256, 32)     1568      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 Convolution_Layer2 (Conv2D)  (None, 128, 128, 64)     32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 64)       0         
 2D)                                                             
                                                                 
 Convolution_Layer3 (Conv2D)  (None, 64, 64, 64)       65600     
                                                                 
 Flatten_layer (Flatten)     (None, 262144)            0

In [ ]:
def train_model(model, XY_train, XY_val, batch_size, epochs):
    # TODO: train the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    # Lets add early stopping if the val loss doesn't improve after 3 epochs
    # early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
    checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_loss', save_best_only=True, mode='min')

    # Using 12-20 epochs and including validation data. Also added verbose to monitor output with early stopping. 
    # Can use early stopping as well buthave it commented out for this instance
    # history = model.fit(X_train, Y_train, epochs=20, batch_size=64,verbose=1, validation_data=(X_val, Y_val), callbacks=[early_stopping, checkpoint])
    history = model.fit(XY_train, 
                        epochs=epochs, batch_size=batch_size, verbose=1, 
                        validation_data=(XY_val), validation_steps=XY_val.samples // batch_size,
                        callbacks=[checkpoint])

    return model, history

model, history = train_model(model, train_gen, val_gen, batch_size=128, epochs=12)

Epoch 1/20
100/100 [==============================] - ETA: 0s - loss: 1.6991 - accuracy: 0.4262 

In [ ]:
def eval_model(model, X_test, Y_test):
    # TODO: evaluate the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
    model.load_weights('best_weights.h5')
    test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=1)

    return test_loss, test_accuracy

## You may use this space (and add additional cells for exploration)

model = build_model()
model = compile_model(model)
model, history = train_model(model, X_train, Y_train, X_val, Y_val)
test_loss, test_accuracy = eval_model(model, X_test, Y_test)

# added this line as want to see what the predicted classification for an input of X_test is
output = model.predict(X_test)

print("\nX_test Accuracy = ", test_accuracy*100,"%")
print("X_test Loss = ", test_loss*100,"%")

#Everything after this is either depracated or experimental

In [ ]:
#you don't need this as the generator automatically one hot encoded the labels

def one_hot_data(train_data, val_data):

  _ , batch_labels = next(train_data)
  print(batch_labels)
  train_labels = train_data.classes
  classes = train_data.num_classes
  one_hot_train_labels = to_categorical(train_labels, num_classes=classes)
   
  val_labels = train_data.classes
  one_hot_val_labels = to_categorical(val_labels, num_classes=classes)
   



  return one_hot_train_labels, one_hot_val_labels


Y_train, Y_val = one_hot_data(train_gen, val_gen)
print(Y_train.shape)
print(Y_train[12713])

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(12714, 11)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
